# 01 Notebook

# DELETE THIS SECTION:

In [ ]:
# import pandas as pd
# from hdfs import InsecureClient

# from datetime import datetime

# df = pd.read_csv("./bigdata.csv")

# BATCH_SIZE = 10_000
# df = df.iloc[:BATCH_SIZE].drop(columns=["description"])

# print("SZIE: ", df.shape)

# # HDFS connection parameters
# hdfs_host = '192.168.64.102'
# hdfs_port = 9870  # Standard HDFS port
# hdfs_user = 'enricogoerlitz'

# # Convert DataFrame to CSV string
# csv_content = df.to_csv(index=False)

# # Create an HDFS client
# hdfs_client = InsecureClient(f'http://{hdfs_host}:{hdfs_port}', user=hdfs_user)

# MAX_SIZE = 10_000_000
# ITTERATIONS = int(MAX_SIZE / BATCH_SIZE) + 1

# print(f"ITTERATIONS: {ITTERATIONS}")
# start_time = datetime.now()
# for i in range(1, ITTERATIONS):
#     # Write the CSV content to HDFS
#     hdfs_path = f"/hive/bigdata/file{i}.csv"
#     with hdfs_client.write(hdfs_path, overwrite=True) as hdfs_file:
#         hdfs_file.write(csv_content)

#     print(f"CSV file was successfully stored in HDFS at {hdfs_path}.")

# end_time = datetime.now()
# duration_time = (end_time - start_time).total_seconds()
# print(f"TOTAL TIME: {duration_time}s")

In [9]:
import pandas as pd

from hdfs import InsecureClient
from datetime import datetime


hdfs_host = '192.168.64.102'
hdfs_port = 9870
hdfs_user = 'enricogoerlitz'
hdfs_client = InsecureClient(f'http://{hdfs_host}:{hdfs_port}', user=hdfs_user)

iterator_db_employee_pay = pd.read_csv("./database/bigdatafiles/datev.dbo.employee_pay.csv", chunksize=10_000, sep="|")

start_time = datetime.now()

chunk: pd.DataFrame
for i, chunk in enumerate(iterator_db_employee_pay):
    parquet_content = chunk.to_parquet(index=False)

    hdfs_path = f"/hive/test/datev/employee_pay/file{i}.parquet"
    with hdfs_client.write(hdfs_path, overwrite=True) as hdfs_file:
        hdfs_file.write(parquet_content)

    print(f"PARQUET file was successfully stored in HDFS at {hdfs_path}.")

end_time = datetime.now()
duration_time = (end_time - start_time).total_seconds()
print(f"TOTAL TIME: {duration_time}s")  # 15_142_128 Records => 809.731399s | 13m 29.7s


PARQUET file was successfully stored in HDFS at /hive/test/datev/employee_pay/file0.parquet.
PARQUET file was successfully stored in HDFS at /hive/test/datev/employee_pay/file1.parquet.
PARQUET file was successfully stored in HDFS at /hive/test/datev/employee_pay/file2.parquet.
PARQUET file was successfully stored in HDFS at /hive/test/datev/employee_pay/file3.parquet.
PARQUET file was successfully stored in HDFS at /hive/test/datev/employee_pay/file4.parquet.
PARQUET file was successfully stored in HDFS at /hive/test/datev/employee_pay/file5.parquet.
PARQUET file was successfully stored in HDFS at /hive/test/datev/employee_pay/file6.parquet.
PARQUET file was successfully stored in HDFS at /hive/test/datev/employee_pay/file7.parquet.
PARQUET file was successfully stored in HDFS at /hive/test/datev/employee_pay/file8.parquet.
PARQUET file was successfully stored in HDFS at /hive/test/datev/employee_pay/file9.parquet.
PARQUET file was successfully stored in HDFS at /hive/test/datev/emplo

In [5]:
from sqlalchemy import create_engine
import pandas as pd

from hdfs import InsecureClient
from datetime import datetime

MYSQL_USERNAME = "hadoop"
MYSQL_PASSWORD = "hadoop_pw"
MYSQL_HOST = "192.168.64.102:3306"
MYSQL_DB = "mysqltestdb"

MYSQL_TABLENAME = "bigdatatable2"

iterator_db_employee_pay = pd.read_csv("./database/bigdatafiles/datev.dbo.employee_pay.csv", chunksize=10_000, sep="|")

engine = create_engine(f'mysql://{MYSQL_USERNAME}:{MYSQL_PASSWORD}@{MYSQL_HOST}/{MYSQL_DB}')

start_time = datetime.now()

chunk: pd.DataFrame
for i, chunk in enumerate(iterator_db_employee_pay):
    chunk.to_sql(
        name="bigdatatable2",
        con=engine,
        if_exists="append",
        index=False
    )
    print(f"CHUNK was successfully stored in MYSQL at {MYSQL_TABLENAME} Loop: {i}.")

end_time = datetime.now()
duration_time = (end_time - start_time).total_seconds()
print(f"TOTAL TIME: {duration_time}s")  # 15_142_128 Records => 

OperationalError: (MySQLdb.OperationalError) (2003, "Can't connect to MySQL server on '192.168.64.102:3306' (60)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)